In [1]:
from __future__ import absolute_import, division, print_function


import os
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.contrib.eager as tfe

import numpy as np
import pandas as pd
import sklearn as skl
tf.enable_eager_execution()



C:\Users\scnef\AppData\Local\conda\conda\envs\py36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:

def parse_csv(line):
  example_defaults = [[0], [0.], [0.], [0.]]  # sets field types
  parsed_line = tf.decode_csv(line, example_defaults)
    
  # First 4 fields are features, combine into single tensor
  features = tf.reshape(parsed_line[1:], shape=(3,))
  # Last field is the label
  label = tf.reshape(parsed_line[0], shape=())
  return features, label

train_dataset_fp = 'C:\\Users\\scnef\github\\taller_tensorflow\\talleres\\dia_1\\data\\data1_Default_train.csv'
train_dataset = tf.data.TextLineDataset(train_dataset_fp)
train_dataset = train_dataset.skip(1)             # skip the first header row
train_dataset = train_dataset.map(parse_csv)      # parse each row
train_dataset = train_dataset.shuffle(buffer_size=100000)  # randomize
train_dataset = train_dataset.batch(32)
features, label = iter(train_dataset).next()
print("example features:", features[0])
print("example label:", label[0])

example features: tf.Tensor([    0.     745.73 60180.56], shape=(3,), dtype=float32)
example label: tf.Tensor(0, shape=(), dtype=int32)


In [3]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation="relu", input_shape=(3,)),  # input shape required
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(2)
])

In [4]:
def loss(model, x, y):
  y_ = model(x)
  return tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)


def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets)
  return tape.gradient(loss_value, model.variables)

In [5]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

In [10]:
## Note: Rerunning this cell uses the same model variables

# keep results for plotting
train_loss_results = []
train_accuracy_results = []

num_epochs = 100

for epoch in range(num_epochs):
  print(epoch)  
  epoch_loss_avg = tfe.metrics.Mean()
  epoch_accuracy = tfe.metrics.Accuracy() 
  recall = []
  # Training loop - using batches of 32
  for x, y in train_dataset:
    # Optimize the model
    grads = grad(model, x, y)
    #print(tf.train.get_or_create_global_step())
    optimizer.apply_gradients(zip(grads, model.variables))
                              #global_step=tf.train.get_or_create_global_step())
    #print(tf.train.get_or_create_global_step())
    
    # Track progress
    epoch_loss_avg(loss(model, x, y))  # add current batch loss
    # compare predicted label to actual label
    epoch_accuracy(tf.argmax(model(x), axis=1, output_type=tf.int32), y)
    atun=skl.metrics.recall_score(y,tf.argmax(model(x), axis=1, output_type=tf.int32))
    
  # end epoch
  train_loss_results.append(epoch_loss_avg.result())
  train_accuracy_results.append(epoch_accuracy.result())
  recall.append(atun)  
  if epoch % 10 == 0:
    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}, Recall: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result(),
                                                                                atun))

0
Epoch 000: Loss: 0.142, Accuracy: 96.775%, Recall: 0.000%
1
2
3
4
5
6
7
8
9
10
Epoch 010: Loss: 0.142, Accuracy: 96.775%, Recall: 0.000%
11
12
13
14
15
16
17
18
19
20
Epoch 020: Loss: 0.142, Accuracy: 96.775%, Recall: 0.000%
21
22
23
24
25
26
27
28
29
30
Epoch 030: Loss: 0.142, Accuracy: 96.775%, Recall: 0.000%
31
32
33
34
35
36
37
38
39
40
Epoch 040: Loss: 0.142, Accuracy: 96.775%, Recall: 0.000%
41
42
43
44
45
46
47
48
49
50
Epoch 050: Loss: 0.142, Accuracy: 96.775%, Recall: 0.000%
51
52
53
54
55
56
57
58
59
60
Epoch 060: Loss: 0.144, Accuracy: 96.775%, Recall: 0.000%
61
62
63
64
65
66
67
68
69
70
Epoch 070: Loss: 0.144, Accuracy: 96.775%, Recall: 0.000%
71
72
73
74
75
76
77
78
79
80
Epoch 080: Loss: 0.142, Accuracy: 96.775%, Recall: 0.000%
81
82
83
84
85
86


KeyboardInterrupt: 